In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from random import gauss, randrange, random

In [ ]:
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

## Modelos Simples de Séries Temporais

* Selecionar um modelo probabilístico __adequado__ para os dados é uma das partes mais __importantes__ da análise de séries temporais;
* Geralmente, é __suposto__ que cada __observação x__ seja um valor resultante de uma determinada __variável aleatória X__;
* Um __modelo__ de série temporal para um determinado dado observado $x_{t}$ é a especificação da composição de distribuições da sequência da variável aleatória $X_{t}$ em que $x_{t}$ é uma observação;


### Modelos Simples com média zero

* Ruído I.I.D (Independente e Identicamente Distribuído);
* Processo Binário;
* Random Walk;


#### Ruído I.I.D 

* Modelo mais simples;
* Sem componentes de tendência e sazonalidade;
* As observações são resultados de variáveis aleátorias i.i.d com média zero;
* Não existe dependência entre as observações;
* Apesar de não ser interessante para predição, é um modelo que ajuda na construção e entendimento de modelos mais complexos;

In [ ]:
## Série gerada por Ruído I.I.D Gaussiano 
dados_iid = [gauss(0.0, 1.0) for i in range(500)]
serie_iid = pd.Series(dados_iid)

In [ ]:
serie_iid.plot()
plt.show()

#### Processo Binário
* As observações só podem assumir dois valores possivéis

In [ ]:
serie_vitorias = pd.DataFrame([1,0,0,1,0,1,0,1,1,1,1,1,1,1,0,1,0,1,0,0])

In [ ]:
serie_vitorias.columns = ['Resultado']
serie_vitorias.plot()
plt.show()

#### Random Walk

* Idéia remete a “caminhada do bêbado”. Tomada de vários passos consecutivos, cada qual em uma direção aleatória;
* É obtido através da soma cumulativa de variáveis i.i.d aleatórias: $S_{t} = X_{1} + X_{2}, ..., + X_{t}$
* $X_{t}$ é um ruído i.i.d
* $S_{t} - S_{t-1} = X_{t}$

In [ ]:
## Série gerada por um Random Walk
serie_randomwalk = []
serie_randomwalk.append(np.random.normal(0))
for i in range(1, 1000):
    movement = -1 if random() < 0.5 else 1
    value = serie_randomwalk[i-1] + movement
    serie_randomwalk.append(value)


In [ ]:
serie_randomwalk = pd.Series(serie_randomwalk)
serie_randomwalk.plot()
plt.show()

### Modelos com tendência e sazonalidade
* Algumas séries é perceptível que não podem ser modeladas por modelos simples de média zero;
* São geradas com componentes de tendência e sazonalidade;
* __Tendência__: mudança sistemática na série temporal que não parece ser periódico;
* __Sazonalidade__: comportamento que se repete durante um período; 

$X_{t} = m_{t} + s_{t} + Y_{t}$


###  Modelo com Tendência

$$X_{t} = m_{t} + Y_{t}$$

$m_{t}$ é um componente de tendência e pode ser estimado através do método de mínimo quadrado:

$$m_{t} = a_{0}+a_{1}t+ a_{2}t^2$$

Os valores de $a$ são ajustados para minimizar a função: 
$$\sum_{t=1}^{n} (x_{t} - m_{t})^2$$

In [ ]:
dados = pd.read_excel('vendas_varejo.xlsx')

In [ ]:
dados.head()

In [ ]:
serie_vendas = pd.Series(dados['Indice'].values, index= dados['Data'])

In [ ]:
serie_vendas.plot()

In [ ]:
serie_vendas.loc['2000':'2002'].plot()

In [ ]:
m_m = serie_vendas.rolling(window=12).mean()
m_std = serie_vendas.rolling(window=12).std() 

orig = plt.plot(serie_vendas, color='blue',label='Original')
mean = plt.plot(m_m, color='red',label='Média')
std = plt.plot(m_std, color='black',label='Desvio')
plt.legend(loc='best')
plt.show()

#### Estimando o componente de tendência 

* Em modelos de machine learning, a tendência adiciona uma complexidade temporal, dificultando para o modelo capturar os padrões e a relação entrada e saída;
* A tendência pode ser utilizada como uma variável adicional em modelos de machine learning

* No entanto, é uma técnica pouca utilizada em machine learning 

In [ ]:
#utilizando regressão linear para modelar tendência
from sklearn.linear_model import LinearRegression


X = [i for i in range(0, len(serie_vendas))] #indices
X = np.reshape(X, (len(X), 1))
y = serie_vendas
modelo = LinearRegression()
modelo.fit(X, y)

In [ ]:
tendencia = modelo.predict(X)

In [ ]:
plt.plot(serie_vendas.values, color='blue',label='Original')
plt.plot(tendencia,color='red', label='Tendência' )
plt.show()

In [ ]:
#eliminando a tendência
serie_sem_tendencia = serie_vendas - tendencia

In [ ]:
m_m = serie_sem_tendencia.rolling(window=12).mean()
mean = plt.plot(m_m, color='red',label='Média')
plt.plot(serie_sem_tendencia, label='Sem tendencia')
plt.legend(loc='best')
plt.show()

#### Estimando o componente de sazonalidade

* Importância para machine learning semelhante ao componente de tendência

In [ ]:
# fit polynomial
from numpy import polyfit

X = [i for i in range(0, len(serie_sem_tendencia))]
y = serie_sem_tendencia.values
degree = 4
coef = polyfit(X, y, degree)
print('Coefficients: %s' % coef)

In [ ]:
curva = []
for i in range(len(X)):
    value = coef[-1]
    for d in range(degree):
        value += X[i]**(degree-d) * coef[d]
    curva.append(value)

In [ ]:
plt.plot(serie_sem_tendencia.values)
plt.plot(curva, color='red', label='sazonalidade')
plt.legend(loc='best')
plt.show()

In [ ]:
#eliminando a sazonalidade
serie_sem_tendencia_sazonalidade = serie_sem_tendencia - curva

In [ ]:
m_m = serie_sem_tendencia_sazonalidade.rolling(window=12).mean()
plt.plot(serie_sem_tendencia_sazonalidade, label='resíduo')
mean = plt.plot(m_m, color='red',label='Média')
plt.legend(loc='best')
plt.show()

#### Aplicando Técnicas de Suavização 

In [ ]:
#Estimando e eliminando a tendência através de média móveis

rolling = serie_vendas.rolling(window=12)
rolling_mean = rolling.mean()

# plot original and transformed dataset
plt.plot(serie_vendas, label='Série Real')
plt.plot(rolling_mean,color='red', label='Média Móveis')
plt.legend(loc='best')
plt.show()

In [ ]:
serie_sem_tendencia = serie_vendas - rolling_mean

In [ ]:
m_m = serie_sem_tendencia.rolling(window=12).mean()
plt.plot(serie_sem_tendencia, label='Sem tendência')
mean = plt.plot(m_m, color='red',label='Média')
plt.legend(loc='best')
plt.show()

In [ ]:
#### Estimando a tendência através de suavização exponencial
rolling = serie_vendas.ewm(alpha =0.5)
rolling_exp = rolling.mean()
plt.plot(serie_vendas, label='Série Real')
plt.plot(rolling_exp,color='red', label='Exponencial')
plt.legend(loc='best')
plt.show()

In [ ]:
#eliminando a tendência
serie_sem_tendencia_exp = serie_vendas - rolling_exp

In [ ]:
m_m = serie_sem_tendencia.rolling(window=12).mean()
plt.plot(serie_sem_tendencia_exp, label='Sem tendência')
mean = plt.plot(m_m, color='red',label='Média')
plt.legend(loc='best')
plt.show()

In [ ]:
 #### Estimando e eliminando a sazonalidade através de média móveis

rolling = serie_sem_tendencia.rolling(window=3)
rolling_mean = rolling.mean()
plt.plot(serie_sem_tendencia, label='Série sem tendência')
plt.plot(rolling_mean,color='red', label='Sazonalidade estimada')
plt.legend(loc='best')
plt.show()

In [ ]:
# eliminando o componente de sazonalidade
serie_sem_tendencia_sazonalidade = serie_sem_tendencia - rolling_mean

In [ ]:
m_m = serie_sem_tendencia_sazonalidade.rolling(window=12).mean()
plt.plot(serie_sem_tendencia_sazonalidade, label='resíduo')
mean = plt.plot(m_m, color='red',label='Média')
plt.legend(loc='best')
plt.show()

#### Utilizando decomposição 

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
decomp = seasonal_decompose(serie_vendas)

In [ ]:
tendencia = decomp.trend
sazonalidade = decomp.seasonal
residuo = decomp.resid

In [ ]:

plt.subplot(411)
plt.plot(serie_vendas, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(tendencia, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(sazonalidade,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residuo, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
plt.plot(tendencia);

#### Analisando o ruído

In [ ]:
import statsmodels.graphics.tsaplots as smt

In [ ]:
serie_sem_tendencia_sazonalidade.dropna(inplace=True) 

In [ ]:
smt.plot_acf(serie_sem_tendencia_sazonalidade, lags=20, alpha=0.05);

In [ ]:
smt.plot_acf(residuo.dropna(), lags=20, alpha=0.05);

#### Aplicando diferenciação

In [ ]:
serie_diff_pd = serie_vendas.diff()

In [ ]:
serie_diff_pd.plot()

In [ ]:
serie_diff_pd.diff().plot()

In [ ]:
smt.plot_acf(serie_diff_pd.dropna(), lags=20, alpha=0.05);

In [ ]:
serie_vendas.diff(12).plot()

In [ ]:
smt.plot_acf(serie_vendas.diff(12).dropna(), lags=20, alpha=0.05);

In [ ]:
smt.plot_acf(serie_vendas, lags=20, alpha=0.05);